<a href="https://colab.research.google.com/github/eunseochu/OB_6th/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EC%84%9D(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **8. 텍스트 분석**

### **5) 감성 분석**

#### **5-1) 감성 분석 소개**
문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법
- 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용
- 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성 수치를 계산하는 방법을 이용
  - 감성 지수는 긍정 감성 지수와 부정 감성 지수로 구성
  - 이들 지수를 합산해 긍정 감성 또는 부정 감성을 결정
- 머신러닝 관점에서 지도학습과 비지도학습 방식으로 나누어 짐
  - 지도학습: 학습 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤, <br/> 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법으로 일반적인 텍스트 기반의 분류와 거의 동일
  - 비지도학습: 'Lexicon'이라는 일종의 감성 어휘 사전을 이용
  - Lexicon은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있으며, <br/> 이를 이용해 문서의 긍정적, 부정적 감성 여부를 판단

#### **5-2) 지도학습 기반 감정 분석 실습 - IMDB 영화평**
영화평의 텍스트를 분석해 감성 분석 결과가 긍정 또는 부정인지를 예측하는 모델 생성
- 유명한 IMDB의 영화 사이트의 영화평을 이용
- 감성 분석이라는 타이틀이 붙었지만, 지도학습 기반 감성 분석의 텍스트 기반의 이진 분류

**데이터 로딩**

In [2]:
import pandas as pd

review_df = pd.read_csv('/content/labeledTrainData.tsv', header=0, sep="\t", quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


- **id**: 각 데이터의 id
- **sentiment**: 영화평(reveiw)의 Sentiment 결과 값(Target Label)
  - 1은 긍정적 평가, 0은 부정적 평가를 의미
- **review**: 영화평의 텍스트

**로드된 데이터**

In [3]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

- HTML 형식에서 추출해 &lt;br/&gt; 태그가 여전히 존재
  - &lt;br/&gt; 문자열은 피처로 만들 필요가 없으므로 삭제

**데이터 사전 처리 html태그 제거 및 숫자문자 제거** <br/>

In [4]:
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />',' ')

# 파이썬의 정규 표현식 모듈인 re를 이용하여 영어 문자열이 아닌 문자는 모두 공백으로 변환 
review_df['review'] = review_df['review'].apply( lambda x : re.sub("[^a-zA-Z]", " ", x) )

- 판다스의 DataFrame/Series는 문자열 연산을 지원하기 위해 str 속성을 이용
  - DataFrame/Series 객체에서 str을 적용하면 다양한 문자열 연산 수행 가능
  - replace()를 str에 적용해 &lt;br/&gt; 태그를 공백으로 모두 변경
- 영어가 아닌 숫자/특수문자 역시 Serntiment를 위한 피처로는 의미가 없어보이므로 모두 공란으로 변경
  - 숫자/특수문자를 찾고 이를 변환하는 것은 정규 표현식을 이용
  - 파이썬의 re 모듈은 편리하게 정규 표현식을 지원
  - 정규 표현식 [^a-zA-Z]의 의미는 영어 대/소문자가 아닌 모든 문자를 찾는 것
  - re.sub("[^a-zA-Z]", "", x)는 영어 대/소문자가 아닌 모든 문자를 찾아서 공란으로 변경
  - 판다스 DataFrame에 re.sub()는 lambda 식을 이용해 적용

**학습/테스트 데이터 분리** <br/>
- 결정 값 클래스인 sentiment 칼럼을 별도로 추출해 결정 값 데이터 세트를 생성
- 원본 데이터 세트에서 id와 sentiment 칼럼을 삭제해 데이터 세트를 생성
- train_test_split()을 이용해 학습용과 테스트용 데이터 세트로 분리

In [5]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape

((2177, 1), (933, 1))

- 학습용 데이터는 17500개의 리뷰, 테그스툥 데이터는 7500개의 리뷰로 구성

**Pipeline을 통해 Count기반 피처 벡터화 및 머신러닝 학습/예측/평가**
- 감상평(Reveiw) 텍스트를 피처 벡터화한 후에 ML 분류 알고리즘ㅇ르 적용해 예측 성능을 측정
- Pipeline 객체를 이용해 두 가지를 한꺼번에 수행
  - Count 벡터화를 적용해 예측 성능을 측정하고, TF-IDF 벡터화를 적용
  - Classifier는 LogisticRegression을 이용
  - 예측 성능 평가는 이진 분류임을 고려해 테스트 데이터 세트의 정확도와 ROC-AUC 모두 측정

**Count 벡터화**

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.  
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8371, ROC-AUC는 0.9128


**TF-IDF 벡터화**

In [7]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8349, ROC-AUC는 0.9235


- 예측 성능이 조금 더 나아짐

#### **5-3) 비지도학습 기반 감정 분석 소개**

**Lexicon** <br/>
비지도 감성 분석은 Lexicon을 기반으로 함
- 결정된 레이블 값이 없을 경우 유용하게 사용될 수 있음
  - 위의 지도 감성 분석은 데이터 세트가 레이블 값을 가지고 있었으나, 많은 감성 분석용 데이터는 결정된 레이블 값을 가지지 않음
  - 그러나 한글을 지원하는 Lexicon은 존재하지 않음
- 일반적으로 어휘집을 의미하지만, 주로 감성만을 분석하기 위해 지원하는 감성 어휘 사전 (감성 사전)
  - **감정 지수(Polarity score)**: 긍정(Positive) 감성 또는 부정(Negative) 감성의 정도를 의미하는 수치
  - 단어의 위치는 주변 단어, 문맥, POS(Part of Speech) 등을 참고해 결정됨
  - 감성 사전을 구현한 대표격은 NLTK 패키지이며, 많은 서브 모듈을 가지고 있고 Lexicon 모듈도 포함됨


**WordNet** <br/>
NLP에서 제공하는 모듈로, 방대한 영어 어휘 사전
- 단순한 어휘 사전이 아닌 **시맨틱 분석을 제공**하는 어휘 사전
  - 시맨틱이란 '문맥상 의미'로 표현할 수 있음
  - 말은 상황에 따라, 문맥에 따라, 화자의 몸짓이나 어조에 따라 다르게 해석될 수 있음
  - 동일한 단어나 문장이라도 다른 환경과 문맥에서는 다르게 표현되거나 이해될 수 있음
  - NLP 패키지는 시맨틱을 프로그램적으로 인터페이스할 수 있는 다양한 방법을 제공
- 다양한 상황에서 같은 어휘라도 다르게 사용되는 어휘의 **시멘틱 정보를 제공**
  - 각각의 품사로 구성된 개별 단어를 Synset(Sets of cognitivie synonyms)이라는 개념을 이용해 표현
  - Synset은 단순한 하나의 단어가 아니라 그 단어가 가지고 있는 문맥, 시맨틱 정보를 제공하는 WordNet의 핵심 개념

**대표적인 감성 사전**
- **SentiWordNet**
  - NLTK 패키지의 WordNet와 유사하게 감성 단어 전용의 WordNet을 구현하며, Synset 개념을 감성 분석에 적용 
  - **WordNet의 Synset별로 3가지 감성 점수(sentiment score)**인 긍정 감성 지수, 부정 감성 지수, 객관성 지수를 할당
  - **긍정 감성 지수는 해당 단어가 감성적으로 얼마나 긍정적**인지, **부정 지수는 얼마나 감성적으로 부정적**인가를 수치로 표현
  - **객관성 지수**는 긍정/부정 감성 지수와 완전히 반대되는 개념으로 단어가 **감성과 관계없이 얼마나 객관적인지**를 수치로 표현
  - 문장별로 단어들의 긍정 감성 지수와 부정 감성 지수를 합산하여 **최종 감성 지수를 계산하고 이에 기반해 감성이 긍정인지 부정인지 결정**
  - 예측 정확도가 그리 높지 않아 잘 사용하지 않음
- **VADER**
  - 주로 소셜 미디어의 텍스트에 대한 감성 분석을 제공하기 위한 패키지
  - 뛰어난 감성 분석 결과를 제공하며, 비교적 빠른 수행 시간을 보장해 대용량 텍스트 데이터에 잘 사용되는 패키지
- **Pattern**
  - 예측 성능 측면에서 가장 주목받는 패키지

#### **5-4) SentiWordNet을 이용한 감성 분석**

##### **5-4-1) WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해**
SentiWordNet은 WordNet 기반의 synset을 이용
- WordNet을 이용하기 위해서는 NLTK를 셋업한 후에 WordNet 서브패키지와 데이터 세트를 내려받아야 함

**NLTK의 모든 데이터 세트와 패키지 다운로드**

In [8]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

**'present' 단어에 대한 Synset 추출**
- Wordnet의 synsets()는 파라미터로 지정된 단어에 대해 WordNet에 등재된 모든 Synset 객체 반환

In [9]:
from nltk.corpus import wordnet as wn

term = 'present'

# 'present'라는 단어로 wordnet의 synsets 생성. 
synsets = wn.synsets(term)
print('synsets() 반환 type :', type(synsets))
print('synsets() 반환 값 갯수:', len(synsets))
print('synsets() 반환 값 :', synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 갯수: 18
synsets() 반환 값 : [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


- synsets() 호출 시 여러 개의 Synset 객체를 가지는 리스트가 반환됨
  - 총 18개의 서로 다른 semantic을 가지는 synset 객체가 반환
- Synset 객체의 파라미터 'present.n.01'은 POS 태그를 나타냄
  - present는 의미, n은 명사 품사
  - 01은 present가 명사로서 가지는 의미가 여러 가지 있어 이를 구분하는 인덱스

**Synset 객체 속성**
- POS(Part of Speech, 품사), 정의(Definition), 부명제(Lemma) 등으로 시맨틱적인 요소 표현

In [10]:
for synset in synsets :
    print('##### Synset name : ', synset.name(),'#####')
    print('POS :',synset.lexname())
    print('Definition:',synset.definition())
    print('Lemmas:',synset.lemma_names())

##### Synset name :  present.n.01 #####
POS : noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
##### Synset name :  present.n.02 #####
POS : noun.possession
Definition: something presented as a gift
Lemmas: ['present']
##### Synset name :  present.n.03 #####
POS : noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
##### Synset name :  show.v.01 #####
POS : verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name :  present.v.02 #####
POS : verb.communication
Definition: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']
##### Synset name :  stage.v.01 #####
POS : verb.creation
Definition: perform (a play), especially on a stage
Lemmas: ['stage', 'present', 'represen

- Synset('present.n.01')과 Synset('present.n.02')는 명사지만 서로 다른 의미를 가지고 있음
  - Synset('present.n.01')은 POS가 noun.time이며, Definition을 살펴보면 '시간적인 의미로 현재'를 나타냄
  - Synset('present.n.02')는 POS가 noun.possession이며, Definition은 '선물'
- Synset('show.v.01')은 동사로서 POS가 verb.perception이며, Definition은 '관객에게 전시를 보여주다'
- Synset은 하나의 단어가 가질 수 있는 여러 가지 시맨틱 정보를 개별 클래스로 나타냄

**WordNet의 path_similarity() 메서드** <br/>
WordNet은 어떤 어휘와 다른 어휘 간의 관계를 유사도로 나타낼 수 있음
- synset 객체는 단어 간의 유사도를 나타내기 위해 path_similarity() 메서드 제공

In [12]:
# synset 객체를 단어별로 생성합니다. 
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree , lion , tiger , cat , dog]
similarities = []
entity_names = [ entity.name().split('.')[0] for entity in entities]

# 단어별 synset 들을 iteration 하면서 다른 단어들의 synset과 유사도를 측정합니다. 
for entity in entities:
    similarity = [ round(entity.path_similarity(compared_entity), 2)  for compared_entity in entities ]
    similarities.append(similarity)
    
# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame형태로 저장합니다.  
similarity_df = pd.DataFrame(similarities , columns=entity_names,index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


**SentiWordNet의 Senti_Synset 클래스** <Br/>
senti_synsets()는 WordNet 모듈이라 synsets()와 비슷하게 Senti_Synset 클래스를 리스트 형태로 반환

In [14]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('senti_synsets() 반환 값 갯수:', len(senti_synsets))
print('senti_synsets() 반환 값 :', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 갯수: 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


- 단어의 감성을 나타내는 감성 지수와 객관성을 나타내는 객관성 지수를 지님
- 감성 지수는 다시 긍정 감성 지수와 부정 감성 지수로 나뉨
  - 어떤 단어가 전혀 감성적이지 않으면 객관성 지수는 1이 되고, 감성 지수는 모두 0이 됨

**father와 fabulous의 감성 지수와 객관성 지수**

In [15]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())
print('\n')
fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ',fabulous .pos_score())
print('fabulous 부정감성 지수: ',fabulous .neg_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125


- father는 객관적인 단어로 객관성 지수가 1.0이고 긍정 감성/부정 감성 지수 모두 0
- fabulous는 감성 단어로서 긍정 감성 지수가 0.875, 부정 감성 지수가 0.125

#### **5-4-2) SentiWordNet을 이용한 영화 감상평 감성 분석** <br/>
IMDB 영화평 감성 분석을 SentiWordNet Lexcion 기반으로 수행

**SentiWordNet을 이용해 감성 분석을 수행하는 순서** <br/>
SentiWordNet을 이용하기 위해서 WordNet을 이용해 문서를 다시 단어로 토큰화한 뒤 어근 추출과 품사 태깅 적용
- 문서(Document)를 문장(Sentence) 단위로 분해
- 다시 문장을 단어(Word) 단위로 토큰화하고 품사 태깅(POS Tagging)
- 품사 태깅된 단어 기반으로 synset 객체와 senti_synset 객체를 생성
- Sneti_synset에서 긍정 감성/부정 감성 지수를 구하고 이를 모두 합산해 <br/> **특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정**

**품사 태깅 수행**

In [16]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return 

**Polartiy Score 합산 함수** <br/>
문서를 문장 -> 단어 토큰 -> 품사 태깅 후에 SentiSynset 클래스를 생성하고 Polartiy Score 합산
- 긍정 감성 지수와 부정 감성 지수를 모두 합한 **총 감성 지수가 0 이상일 경우 긍정 감성, 그렇지 않을 경우 부정 감성**

In [17]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word , tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue                   
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1
    
    return 0

**긍정 및 부정 감성 예측** <br/>
swn_polarity(text) 함수를 IMDB 감상평의 개별 문서에 적용
- 판다스의 apply lambda 구문을 이용해 개별 감상평 텍스트에 적용
- 지도학습 기반의 감성 분석에서 생성한 review_df DataFrame을 그대로 이용
  - review_df의 새로운 칼럼으로 'preds'를 추가해 반환된 감성 평가를 담음
  - 실제 감성 평가인 'sentiment' 칼럼과 반환된 결과의 정확도, 정밀도, 재현율 값 모두 측정

In [18]:
review_df['preds'] = review_df['review'].apply( lambda x : swn_polarity(x) )
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

**감성 분석 예측 성능**

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score 
from sklearn.metrics import recall_score, f1_score, roc_auc_score

print(confusion_matrix( y_target, preds))
print("정확도:", accuracy_score(y_target , preds))
print("정밀도:", precision_score(y_target , preds))
print("재현율:", recall_score(y_target, preds))

[[ 926  633]
 [ 444 1107]]
정확도: 0.6536977491961414
정밀도: 0.6362068965517241
재현율: 0.7137330754352031


- 정확도가 약 66.13%, 재현율이 약 70.91%
- 정확도 지표를 포함한 전반적인 성능 평가 지표는 만족스러울만한 수치는 아님

#### **5-5) VADER를 이용한 감성 분석**
VADER는 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexcion
- SentimentIntensityAnalyzer 클래스를 이용해 쉽게 감성 분석 제공
- NLTK 패키지의 서브 모듈로 제공될 수도 있고 단독 패키지로 제공될 수도 있음

**VADER 사용법** <br/>
NLTK 서브 모듈로 SentimentIntensityAnalyzer를 임포트
- 간략하게 IMDB의 감상평 한 개만 감성 분석을 수행

In [20]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


- SentimentIntensityAnalyzer 객체를 생성한 뒤에 문서별로 polarity_scores() 메서드를 호출해 감성 점수를 구함
  - 해당 문서의 감성 점수가 특정 임계값 이상이면 긍정, 그렇지 않으면 부정으로 판단
- SentimentIntensityAnalyzer 객체의 polarity_scores() 메서드는 딕셔너리 형태의 감성 점수 반환
  - 'neg'는 부정 감성 지수, 'neu'는 중립적인 감성 지수, 'pos' 긍정 감성 지수
  - compound는 neg, neu, pos score를 적절히 조합해 -1에서 1 사이의 감성 지수를 표현한 값
- compound score를 기반으로 부정 감성 또는 긍정 감성 여부를 결정
  - 0.1이면 긍정 감성, 그 이하면 부정 감성으로 판단
  - 상황에 따라 임계값을 적절히 조정해 예측 성능을 조절

**VADER을 이용한 IMDB 감성 분석 수행**
- vader_polarity() 함수 새롭게 생성
  - 입력 파라미터로 영화 감상평 텍스트와 긍정/부정을 결정하는 임곗값을 가짐
  - reveiw_df DataFrame의 apply lambda 식을 통해 vader_polarity() 함수를 호출
  - 각 문서별로 감성 결과를 vader_preds라는 review_df의 새로운 칼럼으로 저장
  - 저장된 감성 분석 결과를 기반으로 VADER의 예측 성능을 측정
- 텍스트별로 vader_polarity() 함수를 호출

In [21]:
def vader_polarity(review,threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # compound 값에 기반하여 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환 
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

# apply lambda 식을 이용하여 레코드별로 vader_polarity( )를 수행하고 결과를 'vader_preds'에 저장
review_df['vader_preds'] = review_df['review'].apply( lambda x : vader_polarity(x, 0.1) )
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

**감성 분석 예측 성능**

In [22]:
print('#### VADER 예측 성능 평가 ####')
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score 
from sklearn.metrics import recall_score, f1_score, roc_auc_score

print(confusion_matrix( y_target, vader_preds))
print("정확도:", accuracy_score(y_target , vader_preds))
print("정밀도:", precision_score(y_target , vader_preds))
print("재현율:", recall_score(y_target, vader_preds))

#### VADER 예측 성능 평가 ####
[[ 855  704]
 [ 206 1345]]
정확도: 0.707395498392283
정밀도: 0.6564177647632992
재현율: 0.8671824629271437


- 정확도가 SentiWordNet보다 향상됐고, 특히 재현율은 약 85.06%로 매우 크게 향상됨
- 이외에도 뛰어난 감성 사전으로 pattern 패키지가 존재
- 감성 사전을 이용한 감성 분석 예측 성능은 지도학습 분류 기반의 예측 성능에 비해서는 아직 낮은 수준이지만, <br/> 결정 클래스 값이 없는 상황을 고려한다면 예측 성능에 일정 수준 만족 가능